# Create and Visualize Forecasts from Today

In [1]:
%reload_ext autoreload
%autoreload 2

import torch
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

from rnn import LSTM
from forecast import forecast_pipeline
from utils import format_yahoo_datetimes

### Inputs - change according to what stocks and the horizon you would like to forecast

In [2]:
# Stocks to forecast and forecast horizon
stocks = ['A']
horizon = 10

# Device
device = 'cpu'

In [6]:


# List to store predictions
outputs = []

# Create predictions for each stock
for _, stock in enumerate(stocks):
    # Load model
    kwargs, state = torch.load(f'models/rnn/{stock}/weights/best.pth', map_location=torch.device(device))
    model = LSTM(**kwargs)
    model.load_state_dict(state)
    model.to(device)
    model.eval()

    # format ticker symbol
    stock = stock.replace('.', '-')   

    # tensor to store future predictions
    predictions = torch.rand(1,0,5)   

    # Send empty tensor to device
    if 'cuda' in device:
        predictions = predictions.cuda()

    # Format datetimes to download most recent daat from Yahoo Finance
    now = dt.datetime.now()
    a = dt.datetime(1970,1,1,23,59,59)
    b = dt.datetime(now.year, now.month, now.day, 23, 59, 59)
    c = b - relativedelta(years=5)

    period1 = str(int((c-a).total_seconds()))   # total seconds from today since Jan. 1, 1970 subracting 5 years
    period2 = str(int((b-a).total_seconds()))   # total seconds from today since Jan. 1, 1970

    period1, period2 = format_yahoo_datetimes()

    # Download data
    data = pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/{x}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=true'.format(x=stock, period1=period1,period2=period2))
    data = data[['Date', 'Open', 'High', 'Low', 'Volume', 'Close']]

    forecast = forecast_pipeline(data, model, horizon, device)

    outputs.append(pd.concat([data, forecast], ignore_index=True))

RuntimeError: For batched 3-D input, hx and cx should also be 3-D but got (2-D, 2-D) tensors

In [4]:
%matplotlib inline
sns.set()
plt.style.use('fivethirtyeight')

# Plot each stock
for i, x in enumerate(outputs):
    plt.figure(figsize=(16, 6))
    plt.title(f"{stocks[i]} - {horizon} Day Forecast")
    plt.gcf().autofmt_xdate()
    plt.ylabel("Close")
    plt.xlabel("Time Step")
    plt.plot(x.loc[(len(x.index)-252-horizon):(len(x.index)-horizon-1), "Date"], x.loc[(len(x.index)-252-horizon):(len(x.index)-horizon-1), "Close"], color='tab:blue')
    plt.plot(x.loc[(len(x.index)-horizon-1):,"Date"], x.loc[(len(x.index)-horizon-1):,"Close"], color='lightcoral')
    plt.show()